In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
Wed Aug 25 12:59:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |         

In [3]:
!unzip "/content/drive/MyDrive/MLData/darknet-syringe-data.zip"

Streaming output truncated to the last 5000 lines.
  inflating: darknet/valid/0119_jpg.rf.699fc3726c005efcd370dc97d816144b.txt  
  inflating: darknet/valid/0120_jpg.rf.32517635aa52c3bcd89d9cd874fe4426.jpg  
  inflating: darknet/valid/0120_jpg.rf.32517635aa52c3bcd89d9cd874fe4426.txt  
  inflating: darknet/valid/0120_jpg.rf.88bda57bbce6d2a1b186cd8bc3497158.jpg  
  inflating: darknet/valid/0120_jpg.rf.88bda57bbce6d2a1b186cd8bc3497158.txt  
  inflating: darknet/valid/0123_jpg.rf.1e69d90ab2d340b3534d52756b99a617.jpg  
  inflating: darknet/valid/0123_jpg.rf.1e69d90ab2d340b3534d52756b99a617.txt  
  inflating: darknet/valid/0123_jpg.rf.5440b383a6d167da491232bb8e0664f2.jpg  
  inflating: darknet/valid/0123_jpg.rf.5440b383a6d167da491232bb8e0664f2.txt  
  inflating: darknet/valid/0124_jpg.rf.b130a3277b1a6de3c21a99d1196e75d9.jpg  
  inflating: darknet/valid/0124_jpg.rf.b130a3277b1a6de3c21a99d1196e75d9.txt  
  inflating: darknet/valid/0125_jpg.rf.4bad333a367266b408838a270f1e862f.jpg  
  inflating: 

In [4]:
%cd '/content/darknet'
!make
!chmod +x ./darknet

/content/darknet
make: Warning: File 'Makefile' has modification time 9696 s in the future
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (

In [5]:
#Set up training file directories for custom dataset

%cp train/_darknet.labels data/obj.names
%mkdir data/obj
#copy image and labels
%cp train/*.jpg data/obj/
%cp valid/*.jpg data/obj/

%cp train/*.txt data/obj/
%cp valid/*.txt data/obj/

with open('data/obj.data', 'w') as out:
  out.write('classes = 2\n') ##CHANGE CLASS NUMBER HERE!!
  out.write('train = data/train.txt\n')
  out.write('valid = data/valid.txt\n')
  out.write('names = data/obj.names\n')
  out.write('backup = backup/')

#write train file (just the image list)
import os

with open('data/train.txt', 'w') as out:
  for img in [f for f in os.listdir('train') if f.endswith('jpg')]:
    out.write('data/obj/' + img + '\n')

#write the valid file (just the image list)
import os

with open('data/valid.txt', 'w') as out:
  for img in [f for f in os.listdir('valid') if f.endswith('jpg')]:
    out.write('data/obj/' + img + '\n')

In [6]:
%cp "/content/drive/MyDrive/MLData/yolo-obj.cfg" "/content/darknet/"

In [8]:
!sudo apt install dos2unix
!dos2unix data/obj.data
!dos2unix cfg/yolov3.cfg
!dos2unix data/obj.names
!dos2unix data/train.txt
!dos2unix data/valid.txt
!dos2unix yolov3.cfg
!dos2unix yolo-obj.cfg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
dos2unix is already the newest version (7.3.4-3).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
dos2unix: converting file data/obj.data to Unix format...
dos2unix: converting file cfg/yolov3.cfg to Unix format...
dos2unix: converting file data/obj.names to Unix format...
dos2unix: converting file data/train.txt to Unix format...
dos2unix: converting file data/valid.txt to Unix format...
dos2unix: converting file yolov3.cfg to Unix format...
dos2unix: converting file yolo-obj.cfg to Unix format...


In [ ]:
!./darknet detector train data/obj.data yolo-obj.cfg darknet53.conv.74 -dont_show -map

Streaming output truncated to the last 5000 lines.
 total_bbox = 5360, rewritten_bbox = 0.130597 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.462057), count: 17, class_loss = 264.350769, iou_loss = 7.035400, total_loss = 271.386169 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.111948), count: 1, class_loss = 1158.598511, iou_loss = 2.569092, total_loss = 1161.167603 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 5053.851562, iou_loss = 0.000000, total_loss = 5053.851562 
 total_bbox = 5378, rewritten_bbox = 0.130160 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.452498), count: 11, class_loss = 265.695312, iou_loss = 3.316040, total_loss = 269.011353 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.539889), count: 1, class_loss = 1156.625977, iou_loss = 0.133667, total_lo

In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
!./darknet detector test data/obj.data cfg/yolov3.cfg backup/yolov3_final.weights img-test.jpg -ext_output -dont_show
#imShow('predictions.jpg')

 CUDA-version: 11000 (11020), GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
conv     21       3 x 3/ 1    224 x 224 x   3 ->  224 x 224 x  21 0.057 BF
   1 conv     21       3 x 3/ 2    224 x 224 x  21 ->  112 x 112 x  21 0.100 BF
   2 conv     21       1 x 1/ 1    112 x 112 x  21 ->  112 x 112 x  21 0.011 BF
   3 conv     21       3 x 3/ 1    112 x 112 x  21 ->  112 x 112 x  21 0.100 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 112 x 112 x  21 0.000 BF
   5 conv     21       3 x 3/ 2    112 x 112 x  21 ->   56 x  56 x  21 0.025 BF
   6 conv     21       1 x 1/ 1     56 x  56 x  21 ->   56 x  56 x  21 0.003 BF
   7 conv     21       3 x 3/ 1     56 x  56 x  21 ->   56 x  56 x  21 0.025 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs:  56 x  56 x 